# Headers/Startup

In [4]:
import requests
from urllib.parse import urlencode
import json
import logging
import traceback
from os import environ as os_environ
from sys import stdout

import dotenv

dotenv.load_dotenv()

from datetime import date, datetime, timedelta

import pandas as pd

# for verifying DB updates
from db_engines import mms_db, rprt_db, wh_db as db

tmstmp_fmt = r'%Y-%m-%d %H:%M:%S'
query_date_fmt = r'%Y-%m-%d'
file_name_date_fmt = r'%Y%m%d'

import re
from pathlib import Path

import pandas as pd
from pandas import DataFrame as Df
from pandas import Series as Ser

from numpy.random import randint

today: str = datetime.now().strftime(query_date_fmt)


In [5]:
log_fmt_date_strm = r'%y%m%d|%H%M'
log_fmt_date_file = r'%Y-%m-%d %H:%M:%S'
log_fmt_file = '%(asctime)s [%(name)s,%(funcName)s,%(module)s::%(levelname)s]>>%(message)s'
log_fmt_strm = '\x1b[32m%(asctime)s[%(name)s %(levelname)s]\x1b[0m >> %(message)s'

logger = logging.getLogger(os_environ['PRMDIA_MM_LOGNAME'])
hdlr = logging.StreamHandler(stdout)
hdlr.setFormatter(logging.Formatter(fmt=log_fmt_strm, datefmt=log_fmt_date_strm))
# hdlr.setLevel(logging.DEBUG)
logger.addHandler(hdlr)
logger.setLevel(logging.INFO)

In [6]:
# output config
from sqlalchemy.dialects.postgresql import BIGINT, INTEGER, ARRAY, TIMESTAMP, VARCHAR


client_lead_cols = json.loads(Path('client_lead_cols.json').read_text())

type_map = {
    'BIGINT': BIGINT,
    'INT': INTEGER,
    'VARCHAR[]': ARRAY(VARCHAR),
    'TIMESTAMP': TIMESTAMP,
}

dtype = {
    k: type_map[d['dtype']]
    for k, d in client_lead_cols.items()
    if d['dtype']
}



In [7]:
# VICI CONFIGS
base_url = 'http://10.1.10.20/vicidial/non_agent_api.php'
static_params: dict[str, str|int|bool] = {
    'user': 6666,
    'pass': 'RedLakeSky3501',
    'function': 'add_lead',
    'custom_fields': 'Y',
    'duplicate_check': 'DUPLIST30DAY',
    'source': __name__
}
var_params: dict[str, str|int|bool] = {
    'list_id': None,
    'phone_number': None,
    'STA': None
}
cmpgn_lists: dict = {
    int(k): int(v) for k, v in
    json.loads(
        Path('campaign_lists.json')
        .read_text()
    ).items()
}


In [8]:
word_list_f = [
    'surgeon',
    'forward',
    'ministry',
    'enhance',
    'peasant',
    'error',
    'final',
    'communist',
    'executrix',
    'memorial',
    'television',
    'bother',
    'advertise',
    'silver',
    'unity',
    'corruption',
    'lazy',
    'pigeon',
    'viable',
    'warrant',
    'response',
    'abridge',
    'present',
    'basketball',
    'angle',
    'realize',
    'explode',
    'rebellion',
    'demonstration',
    'study',
    'descent',
    'absolute',
    'agree',
    'porter',
    'meeting',
    'nervous',
    'rally',
    'clinic',
    'conservation',
    'housing',
    'relationship',
    'rational',
    'vegetarian',
    'physical',
    'afford',
    'orthodox',
    'merchant',
    'dimension',
    'salad',
    'concept'
]

word_list_l = [
'suppress',
 'wilderness',
 'syndrome',
 'benefit',
 'restrain',
 'recognize',
 'buttocks',
 'tycoon',
 'export',
 'economics',
 'indoor',
 'prevalence',
 'suffer',
 'decide',
 'global',
 'exact',
 'reputation',
 'menu',
 'inspiration',
 'literature',
 'sodium',
 'study',
 'dimension',
 'clearance',
 'module',
 'collar',
 'engineer',
 'cemetery',
 'understand',
 'predator',
 'award',
 'reveal',
 'dentist',
 'colorful',
 'mercy',
 'player',
 'include',
 'pigeon',
 'publicity',
 'glacier',
 'piano',
 'exposure',
 'compact',
 'opponent',
 'jelly',
 'color',
 'biology',
 'convention',
 'failure',
 'ivory'
]

In [14]:
# load to vici
# ***ED 105, Neuro 106, and Knee/WtLoss 107 ONLY UNTIL THE OTHER LISTS ARE READY***

list_id = 1008

state_list = ['WI', 'NJ', 'PA', 'IL', 'IN']

# where there is no delivery code
for r in range(50):
    ph: int = 6304189955
    cp = '0000'
    
    # caller state
    st: str = state_list[randint(0,4)]

    args = {
        'list_id': list_id,
        'phone_number': ph,
        'STA': st
    }
    
    # combine static args with variable args
    u = f"{base_url}?{urlencode(static_params | args)}"

    logger.debug(f"{u}")

    resp = requests.get(u)
    logger.info(f"ATT Orig. Ph.: {ph} Campaign ID: {cp} LIST ID: {list_id} ATT state: {st} >> {resp}")
    resp_log = f"resp: {resp} >> {resp.text}".replace('\n', '-|')
    if re.findall('ERROR', resp.text):
        logger.error(f"{resp_log}")
    else: logger.debug(resp_log)




221201|1336[med_mstr INFO] >> ATT Orig. Ph.: 6304189955 Campaign ID: 0000 LIST ID: 1008 ATT state: NJ >> <Response [200]>
221201|1336[med_mstr INFO] >> ATT Orig. Ph.: 6304189955 Campaign ID: 0000 LIST ID: 1008 ATT state: IL >> <Response [200]>
221201|1336[med_mstr ERROR] >> resp: <Response [200]> >> ERROR: add_lead DUPLICATE PHONE NUMBER IN LIST - 6304189955|1008|4328-|
221201|1336[med_mstr INFO] >> ATT Orig. Ph.: 6304189955 Campaign ID: 0000 LIST ID: 1008 ATT state: NJ >> <Response [200]>
221201|1336[med_mstr ERROR] >> resp: <Response [200]> >> ERROR: add_lead DUPLICATE PHONE NUMBER IN LIST - 6304189955|1008|4328-|
221201|1336[med_mstr INFO] >> ATT Orig. Ph.: 6304189955 Campaign ID: 0000 LIST ID: 1008 ATT state: WI >> <Response [200]>
221201|1336[med_mstr ERROR] >> resp: <Response [200]> >> ERROR: add_lead DUPLICATE PHONE NUMBER IN LIST - 6304189955|1008|4328-|
221201|1336[med_mstr INFO] >> ATT Orig. Ph.: 6304189955 Campaign ID: 0000 LIST ID: 1008 ATT state: NJ >> <Response [200]>
221